In [ ]:
#!/usr/bin/python3.8
# coding: utf-8
print("hello from python 3.8")
import os
import platform
import requests
from decimal import Decimal
import time
from configparser import ConfigParser
import datetime
import matplotlib.pyplot as plt
from binance_d import RequestClient
from binance_d.constant.test import *
from binance_d.base.printobject import *
from binance_d import SubscriptionClient
from binance_d.model import *
from binance_d.exception.binanceapiexception import BinanceApiException
from binance_d.model.constant import *


os.environ["TZ"] = 'Asia/Kolkata'
print(platform.system())
if platform.system()!='Windows':
    time.tzset()

c = ConfigParser()
c.read('a.c')
binURL='https://dapi.binance.com'
URL = binURL+'/dapi/v1/ticker/24hr'

request_client = RequestClient(api_key=c.get('m', 'a'), secret_key=c.get('m', 'b'), url=binURL)

sym1='BTCUSD_PERP'
sym2='BTCUSD_210625'
lev=c.getint('m', 'lev')

def get_UnPnL(asset):
    balances = request_client.get_balance()
    for balance in balances:
        if balance.asset==asset:
            return Decimal(balance.crossUnPnl)
def get_balance(asset):
    balances = request_client.get_balance()
    for balance in balances:
        if balance.asset==asset:
            return Decimal(balance.balance)

def get_position(sym):
    positions = request_client.get_position()
    for position in positions:
        if position.symbol==sym:
            return Decimal(position.entryPrice)
def get_position_amount(sym):
    positions = request_client.get_position()
    for position in positions:
        if position.symbol==sym:
            return int(position.positionAmt)



def has_positions():
    pos=0
    positions = request_client.get_position()
    for position in positions:
        if position.positionAmt!=0:
            pos=pos+1
    return pos

def get_spot_price(sym):
    spot_url="https://api.binance.com/api/v3/ticker/price?symbol={}".format(sym)
    r = requests.get(url = spot_url)
    data = r.json()
    print(data['price'])
    return Decimal(data['price'])



asset_bal=get_balance('BTC')*Decimal(0.9)
asset_bal_usd=asset_bal*get_spot_price('BTCUSDT')
lev_asset_bal_usd=asset_bal_usd*lev
qty_contracts=int((lev_asset_bal_usd)/200)

print('One side No.of contracts:{}'.format(qty_contracts))


result = request_client.change_initial_leverage(symbol=sym1, leverage=lev)
result = request_client.change_initial_leverage(symbol=sym2, leverage=lev)

exp_diff=c.getfloat('m', 'exp_diff')
takeProfit_diff=c.getfloat('m', 'takeProfit_diff')
exp_profit = c.getfloat('m', 'exp_profit')
#qty1=c.getint('m', 'qty1')
#qty2=c.getint('m', 'qty2')

qty1=qty_contracts
qty2=qty_contracts
time_delay=c.getint('m', 'time_delay')

print('{},{},{},{}'.format(exp_profit,exp_diff,takeProfit_diff,qty1))


PARAMS1 = {'symbol':sym1}
PARAMS2 = {'symbol':sym2}

signalTrigger=0
signalSide=0

x1 = []
x2 = []
x3 = []
l1=  []
l2=  []
y = []
xPnL=[]
yPnL = []
dictSymIndex={}


imagePath='diff_chart.png'
uPNLImagePath='upnl.png'
signalProfitImagePath='signal.png'
try:
    os.remove(imagePath)
    os.remove(uPNLImagePath)
    os.remove(signalProfitImagePath)
except FileNotFoundError:
    print('FileNotFoundError')
def telegram_bot_sendtext(bot_message,real=0):
    bot_token = '1743327595:AAFvHY9dAVTgFaiYEFDHtd2ZQ2lqvvBbXWs'

    if real==1:
        bot_chatID='-550443241'
        send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message
        response = requests.get(send_text)
    bot_chatID = '-530278791'
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message
    response = requests.get(send_text)
    return response.json()


def telegram_bot_send_photo(image_path, image_caption,real=0):
    bot_token = '1743327595:AAFvHY9dAVTgFaiYEFDHtd2ZQ2lqvvBbXWs'
    url = "https://api.telegram.org/bot%s/sendPhoto" % bot_token

    if real==1:
        bot_chatID='-550443241'
        data = {"chat_id": bot_chatID, "caption": image_caption}
        with open(image_path, "rb") as image_file:
            ret = requests.post(url, data=data, files={"photo": image_file})
    bot_chatID = '-530278791'
    data = {"chat_id": bot_chatID, "caption": image_caption}
    with open(image_path, "rb") as image_file:
        ret = requests.post(url, data=data, files={"photo": image_file})
    return ret.json()

def post_order(sym, qty,orderSide,orderType ):
    try:
        result =request_client.post_order(symbol=sym, quantity=qty,side=orderSide, ordertype=orderType)
        telegram_bot_sendtext(str(result),1)
        telegram_bot_sendtext("{}-{}-{}-{}".format(sym, orderSide, qty, orderType),1)
    except Exception as e:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(e, exc_tb.tb_lineno, exc_type, fname)
        telegram_bot_sendtext(str(e)+'.  line-'+str(exc_tb.tb_lineno),1)



telegram_bot_sendtext('Current open positons={}'.format(has_positions(),0))

for i in range(1, 1000000000):
    try:
        c.read('a.c')
        exp_diff=c.getfloat('m', 'exp_diff')
        takeProfit_diff=c.getfloat('m', 'takeProfit_diff')
        exp_profit = c.getfloat('m', 'exp_profit')
        qty1=qty_contracts
        qty2=qty_contracts
        time_delay=c.getint('m', 'time_delay')

        r = requests.get(url = URL)
        fulldata = r.json()

        j=0
        for symb in fulldata:
            dictSymIndex[symb['symbol']]=j
            j=j+1

        priceChangePercent1=Decimal(fulldata[dictSymIndex[sym1]]['priceChangePercent'])
        lastPrice1=Decimal(fulldata[dictSymIndex[sym1]]['lastPrice'])

        priceChangePercent2=Decimal(fulldata[dictSymIndex[sym2]]['priceChangePercent'])
        lastPrice2=Decimal(fulldata[dictSymIndex[sym2]]['lastPrice'])

        diff=priceChangePercent2-priceChangePercent1

        ct = datetime.datetime.now()
        print('{}  {},{},{}'.format(ct,diff,lastPrice1,lastPrice2 ))
        telegram_bot_sendtext(str(ct)+'    Diff='+str(diff))

        x1.append(diff)
        x2.append(priceChangePercent1)
        x3.append(priceChangePercent2)
        l1.append(lastPrice1)
        l2.append(lastPrice2)
        y.append(ct)

        msg='Price Diff: {};\n {}:  {};\n {}:  {};\n {}:  {};\n {}:  {}'.format(str(diff),str(sym1),str(priceChangePercent1),str(sym2),str(priceChangePercent2),str(sym1),str(lastPrice1),str(sym2),str(lastPrice2) )

        if signalTrigger==0:
            open_pos= has_positions()
            if open_pos!=0:
                signalTrigger=1
                signalPrice=get_position(sym1)
                telegram_bot_sendtext("Already has position with {} @ {}".format(sym1, signalPrice),1)

            if i%10==1:
                plt.plot(y, x1, label = "Diff_Diff",color='green')
                plt.plot(y, x2, label = sym1+"_Diff",color='blue')
                plt.plot(y, x3, label = sym2+"_Diff",color='orange')
                plt.xlabel('Time-->')
                plt.ylabel('Diff %')
                plt.title('Price Diffs')
                #plt.legend()
                plt.grid(True)
                plt.savefig(imagePath)
                plt.show()
                plt.clf()
                telegram_bot_send_photo(imagePath, image_caption=msg)
            if abs(diff) > exp_diff and open_pos==0:
                signalTrigger=1
                telegram_bot_sendtext('Diff: '+ str(diff)+' is greater than exp: '+ msg,1)
                telegram_bot_send_photo(imagePath, str(exp_diff)+"Limit reached",1)

                if diff>0:
                    signalSide=1 #1 for Buy, -1 for Sell
                    signalPrice=lastPrice1
                    signalMessage='Sell {} at {}'.format(sym1, signalPrice)
                    post_order(sym2, qty2,OrderSide.SELL,OrderType.MARKET)
                    post_order(sym1, qty1,OrderSide.BUY,OrderType.MARKET)
                    telegram_bot_sendtext(signalMessage,1)
                else:
                    signalSide=-1 #1 for Buy, -1 for Sell
                    signalPrice=lastPrice1
                    signalMessage='Buy {} at {}'.format(sym1, signalPrice)
                    post_order(sym2, qty2,OrderSide.BUY,OrderType.MARKET)
                    post_order(sym1, qty1,OrderSide.SELL,OrderType.MARKET)
                    telegram_bot_sendtext(signalMessage,1)
        else:
            act_profit=get_UnPnL('BTC')

            satoshi=round(act_profit*100000000,0)
            exp_satoshi=round(exp_profit*100000000,0)
            usd=round(act_profit*lastPrice1, 2)
            exp_usd=round(Decimal(exp_profit)*lastPrice1, 2)
            if satoshi==0:
                signalTrigger=0
                telegram_bot_sendtext('UPnL is ZERO, will look for signal again',1)

            ct = datetime.datetime.now()
            xPnL.append(usd)
            yPnL.append(ct)


            msg = 'UnPNL USD='+str(usd)+'; btc='+str(satoshi)+'/'+str(exp_satoshi)+'; Diff='+str(diff)
            telegram_bot_sendtext(msg,0)
            if i % 2 == 1:
                plt.plot(yPnL, xPnL, label="UnPNL Exp USD: "+str(exp_usd))

                plt.xlabel('Time-->')
                plt.ylabel('PNL')
                plt.title('PNL vs Time')
                #plt.axhline(yPnL= exp_profit, color='blue', linestyle='--', lw=1, label='Exp PNL')
                #plt.legend()
                plt.grid(True)
                plt.savefig(uPNLImagePath)
                #plt.show()
                plt.clf()
                telegram_bot_send_photo(uPNLImagePath, msg,0)


            if act_profit>exp_profit:
                signalTrigger=0
                telegram_bot_sendtext('UPNL={} is greater than Exp PNL={}'.format(act_profit , exp_profit),1)
                telegram_bot_send_photo(uPNLImagePath, msg,0)


                if signalSide==0:
                    positionAmount1=get_position_amount(sym1)
                    positionAmount2=get_position_amount(sym2)
                    if positionAmount1>0:
                        signalSide=1
                    else:
                        signalSide=-1


                if signalSide==1:
                    post_order(sym2, qty2,OrderSide.BUY,OrderType.MARKET)
                    post_order(sym1, qty1,OrderSide.SELL,OrderType.MARKET)

                    telegram_bot_sendtext('Close the positon by Selling {} at {}'.format(sym1, lastPrice1),1)
                    telegram_bot_sendtext('Profit={}-{}={}'.format(lastPrice1,signalPrice,profit),1)
                else:
                    post_order(sym2, qty2,OrderSide.SELL,OrderType.MARKET)
                    post_order(sym1, qty1,OrderSide.BUY,OrderType.MARKET)

                    telegram_bot_sendtext('Close the positon by Buying {} at {}'.format(sym1, lastPrice1),1)
                    telegram_bot_sendtext('Profit={}-{}={}'.format(signalPrice,lastPrice1,profit),1)

                #yPnL=[]
                #xPnL=[]

            if i%2==1:

                fig,ax = plt.subplots()
                ax.plot(y, l1, label =sym1,color="blue")
                ax.plot(y, l2, label =sym2,color="orange")
                ax.set_ylabel('Price',color="blue")
                ax.axhline(y=signalPrice, color='blue', linestyle='--',lw=1, label='Signal Price')
                ax2=ax.twinx()
                ax2.plot(y, x1, label = "K-Factor",color="red")
                ax2.set_ylabel('K-Factor',color="red")
                #ax2.axhline(y=takeProfit_diff, color='red', linestyle='--',lw=1, label='K-Factor Limit')
                #plt.plot(y, x3, label = sym2+"_Diff")
                plt.xlabel('Time-->')
                plt.title(sym1+ ' Price & Diff')

                #plt.legend()
                plt.grid(True)
                plt.savefig(signalProfitImagePath)
                plt.show()
                plt.clf()
                telegram_bot_send_photo(signalProfitImagePath, msg)
            #if abs(diff) < takeProfit_diff:
            if 1==2:
                signalTrigger=0
                telegram_bot_send_photo(uPNLImagePath, 'Current Diff: {} reached Target Diff Level:{}'.format(diff,takeProfit_diff),1)
                signalProfit=(lastPrice1-signalPrice)*signalSide
                print('signalProfit: '+str(signalProfit))
                telegram_bot_sendtext('signalProfit: '+str(signalProfit),1)
                telegram_bot_send_photo(signalProfitImagePath, msg,1)
                profit=act_profit
                if diff<0:
                    post_order(sym1, qty1,OrderSide.BUY,OrderType.MARKET)
                    post_order(sym2, qty2,OrderSide.SELL,OrderType.MARKET)
                    telegram_bot_sendtext('Close the positon by Selling {} at {}'.format(sym1, lastPrice1),1)
                    telegram_bot_sendtext('Profit={}-{}={}'.format(lastPrice1,signalPrice,profit),1)
                else:
                    post_order(sym1, qty1,OrderSide.SELL,OrderType.MARKET)
                    post_order(sym2, qty2,OrderSide.BUY,OrderType.MARKET)
                    telegram_bot_sendtext('Close the positon by Buying {} at {}'.format(sym1, lastPrice1),1)
                    telegram_bot_sendtext('Profit={}-{}={}'.format(signalPrice,lastPrice1,profit),1)
                if profit>0:
                    telegram_bot_sendtext('Congrats :). Your Profit is {}'.format(profit),1)
                else:
                    telegram_bot_sendtext('Sorry. Your Loss is {}'.format(profit),1)
    except Exception as e:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(e, exc_tb.tb_lineno, exc_type, fname)
        telegram_bot_sendtext(str(e)+'.  line-'+str(exc_tb.tb_lineno),0)

    time.sleep(time_delay)
